This notebook was created to test ways to vectorize some of the code

In [3]:
import os
import pandas as pd
import numpy as np

#from sgp4.functions import jday
#from sgp4.conveniences import jday_datetime
from datetime import datetime, timezone

In [4]:
df = pd.read_pickle(os.environ['my_home_path'] + '/data/space-track-gp-hist-sample/raw_compiled/train.pkl.gz')

In [5]:
def jday_tran(x):
    jd = (367.0 * x.year
         - 7 * (x.year + ((x.month + 9) // 12.0)) * 0.25 // 1.0
           + 275 * x.month / 9.0 // 1.0
           + x.day
         + 1721013.5)
    fr = (x.second + (x.microsecond * 1e-6) + x.minute * 60.0 + x.hour * 3600.0) / 86400.0;
    return jd, fr

# from sgp4.functions import jday
# def jday_tran(x):
#     return jday(x.year, x.month, x.day, x.hour, x.minute, x.second + x.microsecond * 1e-6)
%time df[['epoch_jd', 'epoch_fr']] = df['EPOCH'].apply(jday_tran).to_list()

Wall time: 852 ms


In [6]:
ref_date = datetime.strptime('12/31/1949 00:00:00', '%m/%d/%Y %H:%M:%S')
%time df['epoch_days'] = df['EPOCH'].apply(lambda x: (x-ref_date)/np.timedelta64(1, 'D'))

Wall time: 2.95 s


In [8]:
def epoch_days(x, ref_date):
    return (x-ref_date)/np.timedelta64(1, 'D')
%time df['epoch_days'] = df['EPOCH'].apply(epoch_days, ref_date=datetime.strptime('12/31/1949 00:00:00', '%m/%d/%Y %H:%M:%S'))

Wall time: 3.04 s


In [17]:
ref_date=datetime.strptime('12/31/1949 00:00:00', '%m/%d/%Y %H:%M:%S')
%time df['epoch_days2'] = (df['EPOCH']-ref_date)/np.timedelta64(1, 'D')

Wall time: 1.99 ms


In [18]:
df.head()

,NORAD_CAT_ID,OBJECT_TYPE,OBJECT_NAME,TLE_LINE1,TLE_LINE2,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,BSTAR,INCLINATION,RA_OF_ASC_NODE,ECCENTRICITY,ARG_OF_PERICENTER,MEAN_ANOMALY,MEAN_MOTION,EPOCH,epoch_jd,epoch_fr,epoch_days,epoch_days2
1,16378,DEBRIS,COSMOS 1375 DEB,1 16378U 82055S 01123.48663893 +.00000215 +0...,2 16378 065.8389 295.0109 0027819 026.0546 334...,0.000002,0.0,0.000287,65.8389,295.0109,0.002782,26.0546,334.1940,13.794142,2001-05-03 11:40:45.603551,2452032.5,0.486639,18751.486639,18751.486639
2,16377,DEBRIS,COSMOS 1375 DEB,1 16377U 82055R 01123.53731133 +.00001096 +0...,2 16377 065.8313 164.8785 0043812 013.9589 346...,0.000011,0.0,0.000947,65.8313,164.8785,0.004381,13.9589,346.2717,13.900540,2001-05-03 12:53:43.698911,2452032.5,0.537311,18751.537311,18751.537311
3,16376,DEBRIS,COSMOS 1375 DEB,1 16376U 82055Q 01123.61712357 +.00001095 +0...,2 16376 065.8317 207.7286 0024633 342.3797 017...,0.000011,0.0,0.001015,65.8317,207.7286,0.002463,342.3797,17.6386,13.862830,2001-05-03 14:48:39.476448,2452032.5,0.617124,18751.617124,18751.617124
4,16375,DEBRIS,COSMOS 1375 DEB,1 16375U 82055P 01123.85656089 +.00000103 +0...,2 16375 065.8295 286.0123 0025131 356.6652 003...,0.000001,0.0,0.000179,65.8295,286.0123,0.002513,356.6652,3.4243,13.792761,2001-05-03 20:33:26.860895,2452032.5,0.856561,18751.856561,18751.856561
5,16369,ROCKET BODY,SL-8 R/B,1 16369U 85116B 01123.86016933 +.00000175 +0...,2 16369 082.9397 043.1990 0037980 065.0715 295...,0.000002,0.0,0.000163,82.9397,43.1990,0.003798,65.0715,295.4380,13.760971,2001-05-03 20:38:38.630112,2452032.5,0.860169,18751.860169,18751.860169
